In [3]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.io as sio

In [68]:
class EnsemblePursuit():
    def __init__(self):
        self.nr_of_ensembles=None
        self.current_v=np.zeros(1)
        self.current_u=np.zeros(1)
        self.U=np.zeros(1)
        self.V=np.zeros(1)
        
    def calculate_cost_delta(self):
        #print(np.dot(self.current_v.T,self.current_v))
        cost_delta=np.clip(self.current_v.T@self.X,0,None)/np.dot(self.current_v.T,self.current_v)[0]-self.lambd
        cost_delta=cost_delta.reshape(self.X.shape[1],1)
        return cost_delta
    
    def fit_one_assembly(self):
        #Have to figure out a good initialization
        choose_neuron_idx=np.random.randint(0,self.X.shape[1],1)
        self.current_u=np.zeros((self.X.shape[1],1))
        self.current_u[choose_neuron_idx,0]=1
        self.current_v=self.X[:,choose_neuron_idx]
        max_delta_cost=1000
        i=0
        while max_delta_cost>0:
            cost_delta=self.calculate_cost_delta()
            excl_ind=np.where(self.current_u.flatten()==1)[0]
            masked_cost_delta=np.ma.array(cost_delta,mask=self.current_u)
            max_delta_neuron=np.ma.argmax(masked_cost_delta,fill_value=-1000)
            max_delta_cost=cost_delta[max_delta_neuron]
            if max_delta_cost>0:
                self.current_v=(self.current_v+self.X[:,max_delta_neuron].reshape(X.shape[0],1))/2
                self.current_u[max_delta_neuron,0]=1
            i+=1
        print(i)
        self.U=np.hstack((self.U,self.current_u))
        self.V=np.vstack((self.V,self.current_v.T))
        
    #def plot_heatmaps(self):
        
        
    def fit_transform(self,X,lambd,n_ensembles=None):
        self.X=X
        self.lambd=lambd
        self.U=np.zeros((self.X.shape[1],1))
        self.V=np.zeros((1,self.X.shape[0]))
        for iteration in range(0,n_ensembles):
            self.fit_one_assembly()
            self.X=(self.X.T-self.current_u@self.current_v.T).T
            #Reset u for new iteration
            self.current_u=np.zeros((self.X.shape[1],1))
        print(self.U.shape)
        print(self.V.shape)
        self.U=self.U[:,1:]
        self.V=self.V[1:,:]
    

In [5]:
X=sio.loadmat('/home/maria/Documents/EnsemblePursuit/natimg2800_M160825_MP027_2016-12-14.mat')['stim']['resp'][0][0]
print(X.shape)

(5426, 11449)


In [69]:
np.random.seed(7)
ep=EnsemblePursuit()
ep.fit_transform(X,0.5,10)

26
47
25
13
4
9
12
11
18
3
(11449, 11)
(11, 5426)


In [70]:
np.random.seed(7)
import cProfile
ep=EnsemblePursuit()
cProfile.run('ep.fit_transform(X,0.5,10)')

26
47
25
13
4
9
12
11
18
3
(11449, 11)
(11, 5426)
         11124 function calls in 9.872 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       10    0.022    0.002    5.361    0.536 <ipython-input-68-472ca1030b7c>:15(fit_one_assembly)
        1    4.511    4.511    9.872    9.872 <ipython-input-68-472ca1030b7c>:40(fit_transform)
      168    5.241    0.031    5.255    0.031 <ipython-input-68-472ca1030b7c>:9(calculate_cost_delta)
        1    0.000    0.000    9.872    9.872 <string>:1(<module>)
      168    0.000    0.000    0.004    0.000 _methods.py:37(_any)
      168    0.000    0.000    0.001    0.000 core.py:1360(getmask)
      168    0.005    0.000    0.034    0.000 core.py:2771(__new__)
      168    0.003    0.000    0.006    0.000 core.py:2908(_update_from)
      168    0.008    0.000    0.016    0.000 core.py:2934(__array_finalize__)
      840    0.003    0.000    0.003    0.000 core.py:3372(__setattr__)
      168